# Decoding Cellphone Signals with a Quantum Computer
This notebook demonstrates

## What is Coordinated Multipoint?

TBD: add description, papers, etc

# Modeling a Network

TBD: explain the King's graph-based network model

The first code cell imports the needed funcions
NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
# General Python packages
import matplotlib.pyplot as plt

# Ocean software packages
import dimod
from dwave.system import DWaveSampler, FixedEmbeddingComposite

# Functions of this repository
from helpers.draw import draw_network
from helpers.filters import apply_filter, apply_filters, compare_signals, create_filter, create_filters, time_filter_instantiation
from helpers.network import configure_network, create_channels, print_network_stats, simulate_signals

# Enable graphics inline
%matplotlib inline 

## Create a Network Graph

TBD: explain the King's graph-based network model

NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
network_size = 4

network, _ = configure_network(lattice_size=network_size)

print_network_stats(network)
draw_network(network)

The next code cell plots the network graph, showing transmitters in red and receivers in green. 

## Create Channels

Between the transmitters and receivers there are channels etc

In [ ]:
channels, channel_power = create_channels(network)
print(f"`channels` is a {channels.shape[0]} by {channels.shape[1]} matrix with power {channel_power}.")

# Decoding CoMP Transmissions

TBD: explain linear filters

## Create Filters
Explain about filters

In [ ]:
filters = create_filters(channels)
print(f"Created filters: {list(filters.keys())}.")

## Simulate Transmissions


In [ ]:
y, transmitted_symbols = simulate_signals(channels, channel_power)                 
print(y.flatten())

## Decode Received Signals

In [ ]:
v = apply_filters(y, filters)
compare_signals(v, transmitted_symbols)

# Scaling Up

Move code to a helpers/function

In [ ]:
time_filter_instantiation(network_size=[5, 10, 15])

# Solving with a Quantum Computer
Some words here

In [ ]:
qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## BQM Representation (Small-Town Problem)

### Create a Network, Filter, and Simulate Signals

In [ ]:
network, embedding = configure_network(lattice_size=16, ratio=1, qpu=qpu)
print_network_stats(network)

channels, channel_power =  create_channels(network)

filter_mf = create_filter(channels, method='matched_filter')
y, transmitted_symbols = simulate_signals(channels, channel_power)  

### Create a BQM

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

### Decode Received Signals



In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

In [ ]:
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)

## Solving Big-City Problems


### Borderline Noise

In [ ]:
import numpy as np

network, embedding = configure_network(lattice_size=16, ratio=1.5, qpu=qpu)
print_network_stats(network)

sampler = FixedEmbeddingComposite(qpu, embedding)


SNR=5

results = {"QPU": [], "MF": []}
for run in range(10):
    #print(run)
    channels, channel_power =  create_channels(network)
    y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)

    bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)
    
    sampleset = sampler.sample(bqm, num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()), 
                           label='Notebook - Coordinated Multipoint')

    results["QPU"].append(round(100*sum(np.array(list(sampleset.first.sample.values())) == transmitted_symbols.flatten())/len(transmitted_symbols)))
    
    filter_mf = create_filter(channels, method='matched_filter', snr_over_nt=SNR)
    v = apply_filter(filter_mf, y)
    results["MF"].append(round(100*sum(v.flatten() == transmitted_symbols.flatten())/len(transmitted_symbols)))


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(results["QPU"], "b*-", label=f"QPU", markersize=5)
ax.plot(results["MF"], "g^-", label=f"MF", markersize=5)
ax.set(xlabel="Seed", ylabel="Success Rate [%]")
plt.suptitle("P16, Tx/Rx=1.5, Anneal Time 200us")

### ??? High Noise & TX Ratio

In [ ]:
network, embedding = configure_network(lattice_size=16, ratio=1.65, qpu=qpu)
print_network_stats(network)

SNR = 4.5

channels, channel_power =  create_channels(network)

filter_mf = create_filter(channels, method='matched_filter', snr_over_nt=SNR)
y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)

bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

In [ ]:
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)